In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import model_from_json, Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:
train_df = pd.read_csv("../../Datasets/train.csv")
test_df = pd.read_csv("../../Datasets/test.csv")

train_df.drop("Unnamed: 0", axis=1,inplace=True)
test_df.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
train_df.index = train_df["Date and time"]
test_df.index = test_df["Date and time"]

train_df.drop('Date and time', axis=1, inplace=True)
test_df.drop('Date and time', axis=1, inplace=True)

In [4]:
train_df

,Gear oil temperature (°C),Front bearing temperature (°C),Rear bearing temperature (°C),Gear oil inlet temperature (°C),Generator bearing front temperature (°C),Generator bearing rear temperature (°C),Rotor bearing temp (°C),Stator temperature 1 (°C)
Date and time,,,,,,,,
01-01-2020 00:00,56.972500,65.880001,65.382499,55.497500,38.277500,36.752500,27.182501,58.377500
01-01-2020 00:10,56.695001,64.602500,64.712498,55.230000,39.145000,37.037499,27.190001,58.952500
01-01-2020 00:20,56.640000,64.740000,64.917499,55.107500,39.897500,37.347500,27.197501,59.660000
01-01-2020 00:30,56.432500,63.137500,63.990000,55.135000,40.625000,37.752500,27.140001,60.550000
01-01-2020 00:40,56.169998,62.037500,63.525000,54.915000,41.252500,38.362500,27.100000,61.694999
...,...,...,...,...,...,...,...,...
31-12-2020 23:10,50.830000,57.390000,59.570001,50.375000,36.480000,36.380000,23.212501,57.970000
31-12-2020 23:20,50.792500,56.595000,58.920000,50.307499,37.410000,36.672500,23.297499,58.472500
31-12-2020 23:30,50.742500,57.007500,59.552501,50.109999,38.177500,37.022499,23.299999,59.207500


In [5]:
test_df

,Gear oil temperature (°C),Front bearing temperature (°C),Rear bearing temperature (°C),Gear oil inlet temperature (°C),Generator bearing front temperature (°C),Generator bearing rear temperature (°C),Rotor bearing temp (°C),Stator temperature 1 (°C)
Date and time,,,,,,,,
01-01-2021 00:00,17.442500,17.300000,15.615000,12.577500,16.700001,10.675000,10.700000,17.779999
01-01-2021 00:10,18.220000,17.172501,15.445000,13.687500,16.535000,10.600000,10.687500,17.600000
01-01-2021 00:20,18.230000,17.102500,15.490000,14.760000,16.427500,10.430000,10.600000,17.532500
01-01-2021 00:30,17.485000,17.035000,15.340000,12.520000,16.320000,10.327500,10.510000,17.495000
01-01-2021 00:40,17.785000,16.885000,15.220000,13.002500,16.200001,10.285000,10.380000,17.400000
...,...,...,...,...,...,...,...,...
16-06-2021 07:40,57.497499,65.087501,66.695001,56.230000,47.552500,47.002500,31.000000,64.980001
16-06-2021 07:50,57.591667,64.636111,66.141666,56.108333,44.544444,45.872222,31.000000,63.244444
16-06-2021 08:00,57.375001,63.017500,65.092500,55.575000,41.740000,44.455000,31.062500,60.907500
